In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, Point


In [ ]:
pass_data = pd.read_csv("/Users/annadaugaard/Desktop/VFF/VFF_analytics_src/data/03_model_data/test_for_streamit.csv")
gps_data=  pd.read_csv("/Users/annadaugaard/Desktop/VFF/preprocessed_data_for_decision_making.csv", index_col=0)
gps_data_home = gps_data[gps_data["Team"]== "home"]
# Step 1: subset pass_data to home team only
pass_data_home = pass_data[pass_data["Team"] == "home"].reset_index(drop=True)
pass_data_home = pass_data_home[pass_data_home["uncertainty"] < 2].reset_index(drop=True)
# Columns to keep from gps_data
gps_columns = ["time", "player_id", "player_num", "x", "y", "spd", "ball_x","ball_y","Team","acceleration", "smoothed_acceleration"]

# Prepare an empty DataFrame for extracted data
extracted_gps_data = pd.DataFrame(columns=gps_columns + ["pass_event_id"])

# Steps 2 & 3: Extract GPS data for each pass event
for idx, row in pass_data_home.iterrows():
    from_player = row["From"]
    start_time = row["Start Time [s]"]
    end_time = row["End Time [s]"]
    
    # Extract relevant gps data
    gps_subset = gps_data_home[
        (gps_data_home["player_num"] == from_player) &
        (gps_data_home["time"] >= start_time) &
        (gps_data_home["time"] <= end_time)
    ][gps_columns].copy()

    # Add pass event identifier
    gps_subset["pass_event_id"] = idx
    
    # Concatenate results
    extracted_gps_data = pd.concat([extracted_gps_data, gps_subset], ignore_index=True)

extracted_gps_data["distance_to_ball"] = np.hypot(
    extracted_gps_data["x"] - extracted_gps_data["ball_x"],
    extracted_gps_data["y"] - extracted_gps_data["ball_y"]
)
low_distance = extracted_gps_data[extracted_gps_data["distance_to_ball"] < 2]


# Assuming df is your DataFrame
df = low_distance.copy()

# Identify chunks by consecutive player_num
df['player_chunk'] = (df['player_num'] != df['player_num'].shift()).cumsum()

# Get index of max acceleration per chunk
max_acceleration_indices = df.groupby('player_chunk')['acceleration'].idxmax() - 10

# Initialize a list of zeros with length 2500
binary_list = [0] * len(extracted_gps_data)



# Set positions from max_acceleration_indices to 1, ensuring indices within bounds
for idx in max_acceleration_indices:
    if 0 <= idx < len(extracted_gps_data):
        binary_list[idx] = 1
        

# Verify the result (optional)
extracted_gps_data["decision_making_point"] = binary_list

# Iterate over each row in extracted_gps_data with decision-making points
decision_points = extracted_gps_data[extracted_gps_data["decision_making_point"] == 1].reset_index()


# Robust overlap area calculation function
def calculate_overlap_area(target_x, target_y, x_ball, y_ball, away_players, angle_degrees=28.41389086074131, circle_radius=2):
    dx, dy = target_x - x_ball, target_y - y_ball
    height = np.hypot(dx, dy)
    if height == 0:
        return 0.0

    trajectory_angle = np.arctan2(dy, dx)
    half_angle_radians = np.radians(angle_degrees / 2)
    base_width = 2 * height * np.tan(half_angle_radians)

    left_x = target_x + (base_width / 2) * np.cos(trajectory_angle + np.pi / 2)
    left_y = target_y + (base_width / 2) * np.sin(trajectory_angle + np.pi / 2)
    right_x = target_x + (base_width / 2) * np.cos(trajectory_angle - np.pi / 2)
    right_y = target_y + (base_width / 2) * np.sin(trajectory_angle - np.pi / 2)

    points = [(left_x, left_y), (right_x, right_y), (x_ball, y_ball)]

    triangle = Polygon(points)
    if not triangle.is_valid or triangle.area == 0:
        return 0.0

    total_overlap_area = sum(
        triangle.intersection(Point(away_x, away_y).buffer(circle_radius)).area
        for away_x, away_y in zip(away_players["x"], away_players["y"])
    )

    return total_overlap_area

# Updated custom scoring function with directional boost
def custom_score(overlap_area, distance_to_ball, ball_direction_x, alpha=0.21378698795024154, beta=0.04521767482654141, gamma=0.961862342433836):
    direction_bonus = gamma if ball_direction_x > 0 else -gamma
    penalty = alpha * overlap_area + beta * (-distance_to_ball)
    return np.exp(-penalty + direction_bonus)

# Placeholder for results
results = []


for idx, row in decision_points.iterrows():
    current_time = row["time"]
    current_player_num = row["player_num"]
    x_ball, y_ball = row["ball_x"], row["ball_y"]

    players_at_time = gps_data[(gps_data["time"] == current_time) & 
                               (gps_data["player_num"] != current_player_num)]

    home_players = players_at_time[players_at_time["Team"] == "home"]
    away_players = players_at_time[players_at_time["Team"] == "away"]

    for _, home_player in home_players.iterrows():
        
        ball_direction_x = x_ball -home_player["x"] 

        distance_to_ball = np.hypot(home_player["x"] - x_ball, home_player["y"] - y_ball)
        
       # if distance_to_ball < 15:
       #     circle_radius = 2
       # elif 15 < distance_to_ball <= 25:
       #     circle_radius =3
        #else:
       #     circle_radius=6
        
        overlap_area = calculate_overlap_area(
                target_x=home_player["x"],
                target_y=home_player["y"],
                x_ball=x_ball,
                y_ball=y_ball,
                away_players=away_players,
                angle_degrees=28.41389086074131,
                circle_radius=2
            )
        # Using custom scoring with directional boost
        score = custom_score(overlap_area, distance_to_ball, ball_direction_x)

        results.append({
            "timestamp": current_time,
            "reference_player_num": current_player_num,
            "target_player_num": home_player["player_num"],
            "score": score,
            "pass_event_id":row["pass_event_id"]
        })

results_df = pd.DataFrame(results)

# Convert to wide format
results_df_wide = results_df.pivot_table(
    index=['timestamp', "pass_event_id",'reference_player_num'],
    columns='target_player_num',
    values='score'
).reset_index()

results_df_wide.columns = [
    f'score_to_player_{int(col)}' if isinstance(col, (int, float)) else col
    for col in results_df_wide.columns
]

# Select columns containing 'score_to_player'
score_columns = [col for col in results_df_wide.columns if 'score_to_player' in col]

# Find the max value per row among score columns
results_df_wide['max_score'] = results_df_wide[score_columns].max(axis=1)

# Find the corresponding player column for each max score
results_df_wide['max_score_player'] = results_df_wide[score_columns].idxmax(axis=1)

# Display the results
print(results_df_wide[['timestamp', 'reference_player_num', 'max_score', 'max_score_player']])


In [ ]:
decision_points_2 = decision_points[decision_points["player_num"] == 2]

In [ ]:
decision_points_2

In [ ]:
# Merge pass data with annotations to only process annotated events
pass_data_annotated = pd.merge(decision_points_2, annotations2, on='pass_event_id', how='inner')

## THE BEST MODEL; SIMPLE MODEL

In [ ]:


# Robust overlap area calculation function
def calculate_overlap_area(target_x, target_y, x_ball, y_ball, away_players, angle_degrees=28.41389086074131, circle_radius=2):
    dx, dy = target_x - x_ball, target_y - y_ball
    height = np.hypot(dx, dy)
    if height == 0:
        return 0.0

    trajectory_angle = np.arctan2(dy, dx)
    half_angle_radians = np.radians(angle_degrees / 2)
    base_width = 2 * height * np.tan(half_angle_radians)

    left_x = target_x + (base_width / 2) * np.cos(trajectory_angle + np.pi / 2)
    left_y = target_y + (base_width / 2) * np.sin(trajectory_angle + np.pi / 2)
    right_x = target_x + (base_width / 2) * np.cos(trajectory_angle - np.pi / 2)
    right_y = target_y + (base_width / 2) * np.sin(trajectory_angle - np.pi / 2)

    points = [(left_x, left_y), (right_x, right_y), (x_ball, y_ball)]

    triangle = Polygon(points)
    if not triangle.is_valid or triangle.area == 0:
        return 0.0

    total_overlap_area = sum(
        triangle.intersection(Point(away_x, away_y).buffer(circle_radius)).area
        for away_x, away_y in zip(away_players["x"], away_players["y"])
    )

    return total_overlap_area

# Updated custom scoring function with directional boost
def custom_score(overlap_area, distance_to_ball, ball_direction_x, alpha=0.21378698795024154, beta=0.04521767482654141, gamma=0.961862342433836):
    direction_bonus = gamma if ball_direction_x > 0 else -gamma
    penalty = alpha * overlap_area + beta * (-distance_to_ball)
    return np.exp(-penalty + direction_bonus)

# Placeholder for results
results = []

# Iterate over each row in extracted_gps_data with decision-making points
decision_points = extracted_gps_data[extracted_gps_data["decision_making_point"] == 1].reset_index()

for idx, row in decision_points.iterrows():
    current_time = row["time"]
    current_player_num = row["player_num"]
    x_ball, y_ball = row["ball_x"], row["ball_y"]

    players_at_time = gps_data[(gps_data["time"] == current_time) & 
                               (gps_data["player_num"] != current_player_num)]

    home_players = players_at_time[players_at_time["Team"] == "home"]
    away_players = players_at_time[players_at_time["Team"] == "away"]

    for _, home_player in home_players.iterrows():
        
        ball_direction_x = x_ball -home_player["x"] 

        distance_to_ball = np.hypot(home_player["x"] - x_ball, home_player["y"] - y_ball)
        
       # if distance_to_ball < 15:
       #     circle_radius = 2
       # elif 15 < distance_to_ball <= 25:
       #     circle_radius =3
        #else:
       #     circle_radius=6
        
        overlap_area = calculate_overlap_area(
                target_x=home_player["x"],
                target_y=home_player["y"],
                x_ball=x_ball,
                y_ball=y_ball,
                away_players=away_players,
                angle_degrees=28.41389086074131,
                circle_radius=2
            )
        # Using custom scoring with directional boost
        score = custom_score(overlap_area, distance_to_ball, ball_direction_x)

        results.append({
            "timestamp": current_time,
            "reference_player_num": current_player_num,
            "target_player_num": home_player["player_num"],
            "score": score,
            "pass_event_id":row["pass_event_id"]
        })

results_df = pd.DataFrame(results)

# Convert to wide format
results_df_wide = results_df.pivot_table(
    index=['timestamp', "pass_event_id",'reference_player_num'],
    columns='target_player_num',
    values='score'
).reset_index()

results_df_wide.columns = [
    f'score_to_player_{int(col)}' if isinstance(col, (int, float)) else col
    for col in results_df_wide.columns
]

# Select columns containing 'score_to_player'
score_columns = [col for col in results_df_wide.columns if 'score_to_player' in col]

# Find the max value per row among score columns
results_df_wide['max_score'] = results_df_wide[score_columns].max(axis=1)

# Find the corresponding player column for each max score
results_df_wide['max_score_player'] = results_df_wide[score_columns].idxmax(axis=1)

# Display the results
print(results_df_wide[['timestamp', 'reference_player_num', 'max_score', 'max_score_player']])



## CHANGEING RADIUS 

In [ ]:
decision_points = extracted_gps_data[extracted_gps_data["decision_making_point"] == 1].reset_index()

# --- Overlap Area Calculation ---
def calculate_overlap_area(target_x, target_y, x_ball, y_ball, away_players, angle_degrees=40, circle_radius=2):
    dx, dy = target_x - x_ball, target_y - y_ball
    height = np.hypot(dx, dy)
    if height == 0:
        return 0.0

    angle = np.arctan2(dy, dx)
    half_angle = np.radians(angle_degrees / 2)
    base = 2 * height * np.tan(half_angle)

    left = (target_x + (base / 2) * np.cos(angle + np.pi/2),
            target_y + (base / 2) * np.sin(angle + np.pi/2))
    right = (target_x + (base / 2) * np.cos(angle - np.pi/2),
             target_y + (base / 2) * np.sin(angle - np.pi/2))

    triangle = Polygon([left, right, (x_ball, y_ball)])
    if not triangle.is_valid or triangle.area == 0:
        return 0.0

    return sum(
        triangle.intersection(Point(x, y).buffer(circle_radius)).area
        for x, y in zip(away_players["x"], away_players["y"])
    )

# --- Custom Scoring ---
def custom_score(overlap_area, distance_to_ball, ball_direction_x, alpha=0.263, beta=0.035, gamma=0.671):
    direction_bonus = gamma if ball_direction_x > 0 else -gamma
    penalty = alpha * overlap_area + beta * (-distance_to_ball)
    return np.exp(-penalty + direction_bonus)


results = []

for idx, row in decision_points.iterrows():
    current_time = row["time"]
    current_player_num = row["player_num"]
    x_ball, y_ball = row["ball_x"], row["ball_y"]

    players_at_time = gps_data[(gps_data["time"] == current_time) & 
                                (gps_data["player_num"] != current_player_num)]
    home_players = players_at_time[players_at_time["Team"] == "home"]
    away_players = players_at_time[players_at_time["Team"] == "away"]

    for _, home_player in home_players.iterrows():
        ball_direction_x = x_ball - home_player["x"]
        distance_to_ball = np.hypot(home_player["x"] - x_ball, home_player["y"] - y_ball)

        if distance_to_ball < 15:
            circle_radius = 2
        elif distance_to_ball <= 25:
            circle_radius = 3
        else:
            circle_radius = 6

        overlap_area = calculate_overlap_area(
            target_x=home_player["x"],
            target_y=home_player["y"],
            x_ball=x_ball,
            y_ball=y_ball,
            away_players=away_players,
            angle_degrees=22.73,
            circle_radius=circle_radius
        )

        score = custom_score(overlap_area, distance_to_ball, ball_direction_x)

        results.append({
            "timestamp": current_time,
            "reference_player_num": current_player_num,
            "target_player_num": home_player["player_num"],
            "score": score,
            "pass_event_id": row["pass_event_id"]
        })

results_df = pd.DataFrame(results)

results_df_wide = results_df.pivot_table(
    index=['timestamp', "pass_event_id", 'reference_player_num'],
    columns='target_player_num',
    values='score'
).reset_index()

results_df_wide.columns = [
    f'score_to_player_{int(col)}' if isinstance(col, (int, float)) else col
    for col in results_df_wide.columns
]

score_columns = [col for col in results_df_wide.columns if 'score_to_player' in col]
results_df_wide['max_score'] = results_df_wide[score_columns].max(axis=1)
results_df_wide['max_score_player'] = results_df_wide[score_columns].idxmax(axis=1)
   


## WEIGHTED TRIANGLE

In [ ]:

# --- Triangle Density ---
def triangle_density_weighted(px, py, a, b, c):
    local_y = py / 3
    return np.exp(-((local_y - 0.15) ** 2) * 10) + np.exp(-((local_y - 0.85) ** 2) * 10)

# --- Weighted Overlap ---
def calculate_weighted_overlap_area(target_x, target_y, x_ball, y_ball, away_players, angle_degrees=40, circle_radius=2):
    dx, dy = target_x - x_ball, target_y - y_ball
    height = np.hypot(dx, dy)
    if height == 0:
        return 0.0

    angle = np.arctan2(dy, dx)
    half_angle = np.radians(angle_degrees / 2)
    base = 2 * height * np.tan(half_angle)

    left = (target_x + (base / 2) * np.cos(angle + np.pi / 2),
            target_y + (base / 2) * np.sin(angle + np.pi / 2))
    right = (target_x + (base / 2) * np.cos(angle - np.pi / 2),
             target_y + (base / 2) * np.sin(angle - np.pi / 2))

    triangle = Polygon([left, right, (x_ball, y_ball)])
    if not triangle.is_valid or triangle.area == 0:
        return 0.0

    total_weighted_area = 0
    for x, y in zip(away_players['x'], away_players['y']):
        point = Point(x, y).buffer(circle_radius)
        intersection = triangle.intersection(point)
        if not intersection.is_empty:
            rel_x, rel_y = x - x_ball, y - y_ball
            if height > 15:
                density = triangle_density_weighted(rel_x, rel_y, np.array([0, 0]), np.array([-1.5, 3]), np.array([1.5, 3]))
                total_weighted_area += intersection.area * density
            else:
                total_weighted_area += intersection.area 

    return total_weighted_area

# --- Scoring ---
def custom_score(overlap_area, distance_to_ball, ball_direction_x, alpha=0.159, beta=0.047, gamma=0.455):
    direction_bonus = gamma if ball_direction_x > 0 else -gamma
    penalty = alpha * overlap_area + beta * (-distance_to_ball)
    return np.exp(-penalty + direction_bonus)

# --- Filter Decision Points ---
decision_points = extracted_gps_data[extracted_gps_data["decision_making_point"] == 1].reset_index()
#decision_points = decision_points[decision_points["player_num"] == 24]



results = []

for idx, row in decision_points.iterrows():
    current_time = row["time"]
    current_player_num = row["player_num"]
    x_ball, y_ball = row["ball_x"], row["ball_y"]

    players_at_time = gps_data[(gps_data["time"] == current_time) & 
                                (gps_data["player_num"] != current_player_num)]
    home_players = players_at_time[players_at_time["Team"] == "home"]
    away_players = players_at_time[players_at_time["Team"] == "away"]


    for _, home_player in home_players.iterrows():
        ball_direction_x = x_ball - home_player["x"]
        distance_to_ball = np.hypot(home_player["x"] - x_ball, home_player["y"] - y_ball)

        overlap_area = calculate_weighted_overlap_area(
            target_x=home_player["x"],
            target_y=home_player["y"],
            x_ball=x_ball,
            y_ball=y_ball,
            away_players=away_players,
            angle_degrees=38.46,
            circle_radius=2
        )

        score = custom_score(overlap_area, distance_to_ball, ball_direction_x)

        results.append({
            "timestamp": current_time,
            "reference_player_num": current_player_num,
            "target_player_num": home_player["player_num"],
            "score": score,
            "pass_event_id": row["pass_event_id"]
        })

results_df = pd.DataFrame(results)

results_df_wide = results_df.pivot_table(
    index=['timestamp', "pass_event_id", 'reference_player_num'],
    columns='target_player_num',
    values='score'
).reset_index()

results_df_wide.columns = [
    f'score_to_player_{int(col)}' if isinstance(col, (int, float)) else col
    for col in results_df_wide.columns
]

score_columns = [col for col in results_df_wide.columns if 'score_to_player' in col]
results_df_wide['max_score'] = results_df_wide[score_columns].max(axis=1)
results_df_wide['max_score_player'] = results_df_wide[score_columns].idxmax(axis=1)

In [ ]:
#decision_points = extracted_gps_data[extracted_gps_data["decision_making_point"] == 1].reset_index()
results_df_wide['max_score_player'] = results_df_wide['max_score_player'].str.extract('score_to_player_(\d+)').astype(int)

#decision_points_24 = decision_points[decision_points["player_num"]==24]
#decision_points_24.to_csv("deciision_points_24.csv")


## EVALUATE RESULTS PLAYER 24

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
only_24 = results_df_wide[results_df_wide["reference_player_num"] == 24]
only_24.reset_index(drop=True, inplace=True)
only_24.index = only_24.index + 1
df_filtered = only_24.drop(index=[3,6,17,20,25,30,34,40,42,44,45,47])
df_filtered.index = df_filtered.index.astype(int)

annotations = pd.read_csv("/Users/annadaugaard/Desktop/VFF/explore/spiller_24_renskrevet.csv",sep=";")
annotations["player_num"] = annotations["player_num"].astype(int)
# Merge with ground truth using to_x == player_num

merged_df = annotations.merge(df_filtered, left_on='Picture_id', right_index=True)

# Filter rows where player_num matches to_x
matching_rows2 = merged_df[merged_df["max_score_player"] == merged_df["player_num"]]
accuracy = len(matching_rows2)/len(annotations["Picture_id"].unique())


# Merge to identify matched entries by Picture_id and Type
merged = pd.merge(annotations, matching_rows2, on=["Picture_id", "Type"], suffixes=('_ann', '_pred'))

# Count occurrences per Type
annotation_counts_model = annotations["Type"].value_counts().rename("Annotated")
prediction_counts_model = merged["Type"].value_counts().rename("Model Predicted")

# Combine into one DataFrame for plotting
comparison_df_model_24 = pd.concat([annotation_counts_model, prediction_counts_model], axis=1).fillna(0)


comparison_df_model_24["Accuracy (%)"] = (comparison_df_model_24["Model Predicted"] / comparison_df_model_24["Annotated"]) * 100

# Modify the x-axis labels to include the percentage
new_labels = [f"{type_}\n{acc:.1f}%" for type_, acc in zip(comparison_df_model_24.index, comparison_df_model_24["Accuracy (%)"])]

# Calculate overall accuracy
total_annotated = comparison_df_model_24["Annotated"].sum()
total_predicted = comparison_df_model_24["Model Predicted"].sum()

overall_accuracy =  len(matching_rows2)/len(annotations["Picture_id"].unique()) * 100

# Plot with new x-axis labels and overall accuracy as xlabel
ax = comparison_df_model_24[["Annotated", "Model Predicted"]].plot(
    kind="bar", figsize=(10, 6), color=["skyblue", "salmon"]
)
ax.set_title("Comparison of Annotated vs Model Predicted Counts by Type player 24")
ax.set_xlabel(f"Type (Overall Accuracy: {overall_accuracy:.1f}%)")
ax.set_ylabel("Count")
ax.set_xticklabels(new_labels, rotation=0)
plt.grid(axis="y")
plt.tight_layout()
plt.show()

In [ ]:
len(annotations["Picture_id"].unique())

# EVALUATE RESULTS PLAYER 2

In [ ]:
merged_df_subset = merged_df[['player_num','Type', 'preference', 'timestamp', 'reference_player_num']]

In [ ]:
merged_df_subset.to_csv("player_2.csv")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

annotations = pd.read_csv("spiller_2_renskrevet.csv",sep=";")
annotations = annotations.drop(index=5)
only_2 = results_df_wide[results_df_wide["reference_player_num"] == 2]
only_2.reset_index(drop=True, inplace=True)
only_2.index = only_2.index + 1
df_2 = only_2.drop(index=[3,5,7,17,25,30,24])
annotations = pd.read_csv("spiller_2_renskrevet.csv",sep=";")
annotations = annotations.drop(index=[5,12])
annotations["player_num"] = annotations["player_num"].astype(int)
# Merge with ground truth using to_x == player_num
merged_df = annotations.merge(df_2, left_on='Picture_id', right_index=True)
# Filter rows where player_num matches to_x
matching_rows2 = merged_df[merged_df["max_score_player"] == merged_df["player_num"]]
len(matching_rows2)/len(annotations["Picture_id"].unique())

# Filter out NaN values in annotations
annotations = annotations.dropna(subset=["Type"])

# Merge to identify matched entries by Picture_id and Type
merged = pd.merge(annotations, matching_rows2, on=["Picture_id", "Type"], suffixes=('_ann', '_pred'))

# Count occurrences per Type
annotation_counts = annotations["Type"].value_counts().rename("Annotated")
prediction_counts = merged["Type"].value_counts().rename("Model Predicted")

# Combine into one DataFrame for plotting
comparison_df_player_2_model= pd.concat([annotation_counts, prediction_counts], axis=1).fillna(0)

comparison_df_player_2_model["Accuracy (%)"] = (comparison_df_player_2_model["Model Predicted"] / comparison_df_player_2_model["Annotated"]) * 100

# Modify the x-axis labels to include the percentage
new_labels = [f"{type_}\n{acc:.1f}%" for type_, acc in zip(comparison_df_player_2_model.index, comparison_df_player_2_model["Accuracy (%)"])]

# Calculate overall accuracy
total_annotated = comparison_df_player_2_model["Annotated"].sum()
total_predicted = comparison_df_player_2_model["Model Predicted"].sum()
overall_accuracy = (total_predicted / total_annotated) * 100

# Plot with new x-axis labels and overall accuracy as xlabel
ax = comparison_df_player_2_decision[["Annotated", "Model Predicted"]].plot(
    kind="bar", figsize=(10, 6), color=["skyblue", "salmon"]
)
ax.set_title("Comparison of Annotated vs Model Predicted Counts by Type player 2")
ax.set_xlabel(f"Type (Overall Accuracy: {overall_accuracy:.1f}%)")
ax.set_ylabel("Count")
ax.set_xticklabels(new_labels, rotation=0)
plt.grid(axis="y")
plt.tight_layout()
plt.show()

In [ ]:
len(matching_rows2)/len(annotations["Picture_id"].unique())

## CALCULATING PLAYER 2 ACTUAL DECISION PERFORMANCE

In [ ]:
decision_points_2 = decision_points[decision_points["player_num"]==2]
decision_points_2.reset_index(drop=True, inplace=True)
decision_points_2.index = decision_points_2.index + 1
df_filtered_player2 = decision_points_2.drop(index=[3,5,7,17,25,30,34])

real_player2_events = pd.read_csv("/Users/annadaugaard/Desktop/VFF/deployment/tracking/test_for_streamit.csv")
pass_data_home = real_player2_events[real_player2_events["Team"] == "home"].reset_index(drop=True)
pass_data_home = pass_data_home[pass_data_home["uncertainty"] < 2].reset_index(drop=True)

annotations2 = pd.read_csv("spiller_2_renskrevet.csv",sep=";")
annotations2 = annotations2.drop(index=[12,5])
annotations2["player_num"] = annotations2["player_num"].astype(int)

merged_df = df_filtered_player2.merge(pass_data_home, left_on='pass_event_id', right_index=True)
merged_df = merged_df.drop(merged_df.columns[0], axis = 1)
merged_df["Picture_id"] = merged_df.index
final_df = annotations2.merge(merged_df, on="Picture_id", how="left")
matching_rows2 = final_df[final_df["player_num_x"] == final_df["To"]]
accuracy = len(matching_rows2)/len(annotations2["Picture_id"])


# Filter out NaN values in annotations
#annotations = annotations2.dropna(subset=["Type"])

# Merge to identify matched entries by Picture_id and Type
merged = pd.merge(annotations2, matching_rows2, on=["Picture_id", "Type"], suffixes=('_ann', '_pred'))

# Count occurrences per Type
annotation_counts = annotations2["Type"].value_counts().rename("Annotated")
prediction_counts = matching_rows2["Type"].value_counts().rename("Model Predicted")

# Combine into one DataFrame for plotting
comparison_df_player_2_decision = pd.concat([annotation_counts, prediction_counts], axis=1).fillna(0)


comparison_df_player_2_decision["Accuracy (%)"] = (comparison_df_player_2_decision["Model Predicted"] / comparison_df_player_2_decision["Annotated"]) * 100

# Modify the x-axis labels to include the percentage
new_labels = [f"{type_}\n{acc:.1f}%" for type_, acc in zip(comparison_df_player_2_decision.index, comparison_df_player_2_decision["Accuracy (%)"])]

# Calculate overall accuracy
total_annotated = comparison_df_player_2_decision["Annotated"].sum()
total_predicted = comparison_df_player_2_decision["Model Predicted"].sum()
overall_accuracy = len(matching_rows2)/len(annotations2["Picture_id"])* 100

# Plot with new x-axis labels and overall accuracy as xlabel
ax = comparison_df_player_2_decision[["Annotated", "Model Predicted"]].plot(
    kind="bar", figsize=(10, 6), color=["skyblue", "salmon"]
)
ax.set_title("Comparison of Annotated vs Actual Decisions by Type player 2")
ax.set_xlabel(f"Type (Overall Accuracy: {overall_accuracy:.1f}%)")
ax.set_ylabel("Count")
ax.set_xticklabels(new_labels, rotation=0)
plt.grid(axis="y")
plt.tight_layout()
plt.show()


# PLAYER 24 ACTUAL DECISION PERFORMANCE

In [ ]:
decision_points_24 = decision_points[decision_points["player_num"]==24]
decision_points_24.reset_index(drop=True, inplace=True)
decision_points_24.index = decision_points_24.index + 1
df_24 = decision_points_24.drop(index=[3,6,17,20,25,30,34,40,42,44,45,47])

real_player24_events = pd.read_csv("/Users/annadaugaard/Desktop/VFF/deployment/tracking/test_for_streamit.csv")
pass_data_home = real_player24_events[real_player24_events["Team"] == "home"].reset_index(drop=True)
pass_data_home = pass_data_home[pass_data_home["uncertainty"] < 2].reset_index(drop=True)
annotations24= pd.read_csv("spiller_24_renskrevet.csv",sep=";")
annotations24["player_num"] = annotations24["player_num"].astype(int)
merged_df_24 = df_24.merge(pass_data_home, left_on='pass_event_id', right_index=True)
merged_df_24 = merged_df_24.drop(merged_df_24.columns[0], axis = 1)

merged_df_24["Picture_id"] = merged_df_24.index
final_df_24 = annotations24.merge(merged_df_24, on="Picture_id", how="left")
matching_rows24 = final_df_24[final_df_24["player_num_x"] == final_df_24["To"]]
accuracy_24 = len(matching_rows24)/len(annotations24["Picture_id"].unique())
# Merge to identify matched entries by Picture_id and Type
merged = pd.merge(annotations24, matching_rows24, on=["Picture_id", "Type"], suffixes=('_ann', '_pred'))

# Count occurrences per Type
annotation_counts_decision = annotations24["Type"].value_counts().rename("Annotated")
prediction_counts_decision = matching_rows24["Type"].value_counts().rename("Model Predicted")

# Combine into one DataFrame for plotting
comparison_df_decision_24 = pd.concat([annotation_counts_decision, prediction_counts_decision], axis=1).fillna(0)

comparison_df_decision_24["Accuracy (%)"] = (comparison_df_decision_24["Model Predicted"] / comparison_df_decision_24["Annotated"]) * 100

# Modify the x-axis labels to include the percentage
new_labels = [f"{type_}\n{acc:.1f}%" for type_, acc in zip(comparison_df_decision_24.index, comparison_df_decision_24["Accuracy (%)"])]

# Calculate overall accuracy
total_annotated = comparison_df_decision_24["Annotated"].sum()
total_predicted = comparison_df_decision_24["Model Predicted"].sum()
overall_accuracy =  len(matching_rows24)/len(annotations24["Picture_id"].unique())*100

# Plot with new x-axis labels and overall accuracy as xlabel
ax = comparison_df_decision_24[["Annotated", "Model Predicted"]].plot(
    kind="bar", figsize=(10, 6), color=["skyblue", "salmon"]
)
ax.set_title("Comparison of Annotated vs Actual decisions by Type player 24")
ax.set_xlabel(f"Type (Overall Accuracy: {overall_accuracy:.1f}%)")
ax.set_ylabel("Count")
ax.set_xticklabels(new_labels, rotation=0)
plt.grid(axis="y")
plt.tight_layout()
plt.show()


# COMPAIRSON OF MODEL AND PLAYERS


In [ ]:
# Plot
accuracy_df = pd.DataFrame({
    "Model Prediction Accuracy": comparison_df_model_24["Accuracy (%)"],
    "Player Decision Accuracy": comparison_df_decision_24["Accuracy (%)"]
})
colors = ["#F79256", "#006a5d"]
ax = accuracy_df.plot(kind='bar', figsize=(10, 6),color=colors)
ax.set_title("Model 3: Accuracy Comparison Relative to Annotations Player 24")
ax.set_xlabel("Pass Category")
ax.set_ylabel("Accuracy (%)")
plt.xticks(rotation=0)
plt.ylim(0, 100)
#plt.grid(axis="y")

# Add percentage labels on top of bars
for container in ax.containers:
    ax.bar_label(container, fmt=' %.1f%%', label_type='edge', padding=3)

plt.tight_layout()
plt.show()

In [ ]:

counts_df = pd.DataFrame({
    "Annotated": comparison_df_decision_24["Annotated"],  # same for both
    "Model": comparison_df_model_24["Model Predicted"],
    "Player": comparison_df_decision_24["Model Predicted"]
})

colors = ["#c7c7c7","#F79256", "#006a5d"]
# Plotting
ax = counts_df.plot(kind='bar', figsize=(10, 6), color=colors)
ax.set_title("Model 3: Count Comparison between Annotations, Model Prediction and Player 24 Decisions")
ax.set_xlabel("Pass Category")
ax.set_ylabel("Count")
plt.xticks(rotation=0)
#plt.grid(axis="y")

# Add count labels on top of bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f', label_type='edge', padding=3)

plt.tight_layout()
plt.show()

In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(18, 6))
colors = {
    "Model Prediction Accuracy": "#1f77b4",  # blue
    "Player Decision Accuracy": "#ff7f0e",   # orange
    "Annotated": "#c7c7c7",                 # grey
    "Model": "#1f77b4",                     # blue
    "Player": "#ff7f0e"                     # orange
}
accuracy_df = pd.DataFrame({
    "Model Prediction Accuracy": comparison_df_model_24["Accuracy (%)"],
    "Player Decision Accuracy": comparison_df_decision_24["Accuracy (%)"]
})

accuracy_df.plot(kind='bar', ax=axes[0], color=[colors["Model Prediction Accuracy"], colors["Player Decision Accuracy"]])
axes[0].set_title("Hit-Percentage per Type: Model vs Player 24")
axes[0].set_xlabel("Type")
axes[0].set_ylabel("Accuracy (%)")
axes[0].set_ylim(0, 100)
#axes[0].grid(axis="y", linestyle="--", alpha=0.7)
axes[0].tick_params(axis='x', rotation=0)
axes[0].legend(loc="upper right", frameon=False)

for container in axes[0].containers:
    axes[0].bar_label(container, fmt='  %.1f%%', label_type='edge', padding=3)

# Count plot
counts_df = pd.DataFrame({
    "Annotated": comparison_df_decision_24["Annotated"],
    "Model": comparison_df_model_24["Model Predicted"],
    "Player": comparison_df_decision_24["Model Predicted"]
})
counts_df.plot(kind='bar', ax=axes[1], color=[colors["Annotated"], colors["Model"], colors["Player"]])
axes[1].set_title("Count per Type: Annotated vs Model vs Player")
axes[1].set_xlabel("Type")
axes[1].set_ylabel("Count")
#axes[1].grid(axis="y", linestyle="--", alpha=0.7)
axes[1].tick_params(axis='x', rotation=0)
axes[1].legend(loc="upper right", frameon=False)

for container in axes[1].containers:
    axes[1].bar_label(container, fmt='%.0f', label_type='edge', padding=3)

plt.tight_layout()
plt.show()

In [ ]:

accuracy_df = pd.DataFrame({
    "Model Prediction Accuracy": comparison_df_player_2_model["Accuracy (%)"],
    "Player Decision Accuracy": comparison_df_player_2_decision["Accuracy (%)"]
})
colors = ["#F79256", "#006a5d"]
ax = accuracy_df.plot(kind='bar', figsize=(10, 6),color=colors)
ax.set_title("Model 1: Accuracy Comparison Relative to Annotations Player 2")
ax.set_xlabel("Pass Category")
ax.set_ylabel("Accuracy (%)")
plt.xticks(rotation=0)
plt.ylim(0, 100)
#plt.grid(axis="y")

# Add percentage labels on top of bars
for container in ax.containers:
    ax.bar_label(container, fmt=' %.1f%%', label_type='edge', padding=3)

plt.tight_layout()
plt.show()

In [ ]:
comparison_df_player_2_decision

In [ ]:

counts_df = pd.DataFrame({
    "Annotated": comparison_df_player_2_decision["Annotated"],  # same for both
    "Model": comparison_df_player_2_model["Model Predicted"],
    "Player": comparison_df_player_2_decision["Model Predicted"]
})
colors = ["#c7c7c7","#F79256", "#006a5d"]
# Plotting
ax = counts_df.plot(kind='bar', figsize=(10, 6), color=colors)
ax.set_title("Model 1: Count Comparison between Annotations, Model Prediction and Player 2 Decisions")
ax.set_xlabel("Pass Category")
ax.set_ylabel("Count")
plt.xticks(rotation=0)
#plt.grid(axis="y")

# Add count labels on top of bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f', label_type='edge', padding=3)

plt.tight_layout()
plt.show()

In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

accuracy_df = pd.DataFrame({
    "Model Prediction Accuracy": comparison_df_player_2_model["Accuracy (%)"],
    "Player Decision Accuracy": comparison_df_player_2_decision["Accuracy (%)"]
})


colors = {
    "Model Prediction Accuracy": "#1f77b4",  # blue
    "Player Decision Accuracy": "#ff7f0e",   # orange
    "Annotated": "#c7c7c7",                 # grey
    "Model": "#1f77b4",                     # blue
    "Player": "#ff7f0e"                     # orange
}


accuracy_df.plot(kind='bar', ax=axes[0], color=[colors["Model Prediction Accuracy"], colors["Player Decision Accuracy"]])
axes[0].set_title("Hit-Percentage per Type: Model vs Player 24")
axes[0].set_xlabel("Type")
axes[0].set_ylabel("Accuracy (%)")
axes[0].set_ylim(0, 100)
#axes[0].grid(axis="y", linestyle="--", alpha=0.7)
axes[0].tick_params(axis='x', rotation=0)
axes[0].legend(loc="upper right", frameon=False)

for container in axes[0].containers:
    axes[0].bar_label(container, fmt='  %.1f%%', label_type='edge', padding=3)

# Count plot
counts_df = pd.DataFrame({
    "Annotated": comparison_df_player_2_decision["Annotated"],
    "Model": comparison_df_player_2_model["Model Predicted"],
    "Player": comparison_df_player_2_decision["Model Predicted"]
})
counts_df.plot(kind='bar', ax=axes[1], color=[colors["Annotated"], colors["Model"], colors["Player"]])
axes[1].set_title("Count per Type: Annotated vs Model vs Player")
axes[1].set_xlabel("Type")
axes[1].set_ylabel("Count")
#axes[1].grid(axis="y", linestyle="--", alpha=0.7)
axes[1].tick_params(axis='x', rotation=0)
axes[1].legend(loc="upper right", frameon=False)

for container in axes[1].containers:
    axes[1].bar_label(container, fmt='%.0f', label_type='edge', padding=3)

plt.tight_layout()
plt.show()